In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas_ta as ta
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, LeakyReLU, TimeDistributed, Input, Activation
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from datetime import datetime
import os
from TrainingProgress import TrainingProgress

main_log_dir = './logs'
main_model_save_dir = './ModelsSave/'
file_path = './Data/xmrusd.csv'
log_dir = os.path.join(main_log_dir)

current_time = datetime.now().strftime("%Y-%m-%d_%H-%M")

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=1e-6)

DELKA_SEKVENCE = 30
BATCH_SIZE = 16
EPOCHS = 1

2024-07-16 07:00:44.805594: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-16 07:00:45.473699: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

df = pd.read_csv(file_path, parse_dates=['time'], date_format=lambda x: pd.to_datetime(x, unit='ms'))
print(df.head(10))

# Adding your indicators
df['EMA_5'] = ta.ema(df['close'], length=5)
df['SMA_15'] = ta.sma(df['close'], length=15)
df['RSI'] = ta.rsi(df['close'], length=14)
stoch = ta.stoch(df['high'], df['low'], df['close'])
df['STOCH_K'] = stoch['STOCHk_14_3_3']
df['STOCH_D'] = stoch['STOCHd_14_3_3']
macd = ta.macd(df['close'])
df['MACD'] = macd['MACD_12_26_9']
df['MACD_SIGNAL'] = macd['MACDs_12_26_9']
df['MACD_HIST'] = macd['MACDh_12_26_9']

#Target, TargetNextClose, and TargetClass
df['TargetNextClose'] = df['close'].shift(-1)
df['Target'] = df['TargetNextClose'] - df['open']
df['TargetClass'] = df['Target'].apply(lambda x: 1 if x > 0 else 0)


df.dropna(inplace=True)
df.reset_index(inplace=True)
df.drop(['index', 'volume', 'time'], axis=1, inplace=True)

print(df.iloc[2000:2006])

scaler = MinMaxScaler(feature_range=(0, 1))
df[df.columns] = scaler.fit_transform(df[df.columns])

train_size = int(len(df) * 0.7)
val_size = int(len(df) * 0.15)
test_size = len(df) - train_size - val_size

train_data = df[:train_size]
val_data = df[train_size:train_size + val_size]
test_data = df[train_size + val_size:]

print(f"Train data shape: {train_data.shape}")
print(f"Validation data shape: {val_data.shape}")
print(f"Test data shape: {test_data.shape}")

            time   open  close   high    low      volume
0  1480530840000  7.649  7.649  7.649  7.649    0.100000
1  1480530900000  7.681  7.681  7.681  7.681    1.000000
2  1480531140000  7.833  7.968  7.968  7.833   51.000000
3  1480531260000  8.000  8.000  8.000  8.000   23.328046
4  1480531380000  8.000  8.000  8.000  8.000   76.671954
5  1480531620000  8.217  8.217  8.217  8.217    0.100000
6  1480531740000  8.206  8.206  8.206  8.206    0.100000
7  1480531800000  8.217  8.218  8.218  8.206    0.300000
8  1480531860000  7.004  8.250  8.250  7.004  100.100000
9  1480531980000  8.500  8.500  8.500  8.500  100.000000
        open   close    high     low      EMA_5     SMA_15        RSI  \
2000  13.000  13.000  13.000  13.000  12.999082  12.980667  63.521744   
2001  12.855  12.855  12.855  12.855  12.951054  12.976200  44.432884   
2002  12.842  12.842  12.842  12.842  12.914703  12.973733  43.180030   
2003  12.849  12.849  12.849  12.849  12.892802  12.969867  44.094128   
2004  12

In [3]:

input_shape = (DELKA_SEKVENCE, train_data.shape[1])

model = Sequential()
model.add(Input(shape=input_shape))
model.add(BatchNormalization())
model.add(LSTM(32, return_sequences=True))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(TimeDistributed(Dense(32)))
model.add(LeakyReLU())
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(TimeDistributed(Dense(1)))
model.add(Activation('linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mean_absolute_error'])

print(model.summary())

checkpoint_callback = ModelCheckpoint(
    filepath=f'{main_model_save_dir}/best_modelV2.epoch{{epoch:02d}}-val_loss{{val_loss:.2f}}.keras',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    save_freq='epoch'
)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

def data_generator(data, seq_length, batch_size):
    data_length = len(data)
    indices = np.arange(data_length - seq_length - 1)
    np.random.shuffle(indices)
    while True:
        X, y = [], []
        for i in indices:
            X.append(data.iloc[i:i + seq_length].values)
            y.append(data.iloc[i + seq_length, -3])
            if len(X) == batch_size:
                yield np.array(X), np.array(y).reshape(-1, 1)
                X, y = [], []

steps_per_epoch = len(train_data) // BATCH_SIZE
validation_steps = len(val_data) // BATCH_SIZE

tensorboard_callback = TrainingProgress(total_epochs=EPOCHS, total_batches=steps_per_epoch, log_dir=log_dir, update_freq=100)

history = model.fit(
    data_generator(train_data, DELKA_SEKVENCE, BATCH_SIZE),
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=data_generator(val_data, DELKA_SEKVENCE, BATCH_SIZE),
    validation_steps=validation_steps,
    callbacks=[tensorboard_callback, checkpoint_callback, early_stopping_callback, reduce_lr],
    verbose=1
)

# Save the final model
print("Saving final model...", flush=True)
model.save(f'{main_model_save_dir}/final_modelV2.keras')

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization             │ (None, 30, 15)         │            60 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 30, 32)         │         6,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 30, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 30, 32)         │         1,056 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 30, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 30, 32)         │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 30, 1)          │            33 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 30, 1)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,549 (29.49 KB)

 Trainable params: 7,391 (28.87 KB)

 Non-trainable params: 158 (632.00 B)

None
      EMA_5    SMA_15       RSI   STOCH_K   STOCH_D      MACD  MACD_SIGNAL  \
0  0.003384  0.003438  0.567465  0.420279  0.240602  0.646170     0.641149   
1  0.003282  0.003399  0.492870  0.374032  0.311534  0.645550     0.640509   
2  0.003054  0.003367  0.456042  0.374032  0.389447  0.644470     0.639751   
3  0.003068  0.003303  0.499212  0.205420  0.317828  0.644204     0.639085   
4  0.002981  0.003212  0.476098  0.157853  0.245768  0.643641     0.638423   

   MACD_HIST  
0   0.496371  
1   0.496220  
2   0.495143  
3   0.495977  
4   0.496025  
56799/56799 ━━━━━━━━━━━━━━━━━━━━ 550s 10ms/step - loss: 0.0223 - mean_absolute_error: 0.0730 - val_loss: 1.7531e-04 - val_mean_absolute_error: 0.0112 - learning_rate: 0.0010
Saving final model...
